In [ ]:
import re

class Node:
    def __init__(self, node_type, value=None, left=None, right=None):
        self.node_type = node_type  # "operator" or "operand"
        self.value = value          # For operands, it could be conditions like "age > 30"
        self.left = left            # Left child for operators
        self.right = right          # Right child for operators

    def __repr__(self):
        if self.node_type == "operand":
            return f"{self.value}"
        else:
            return f"({self.left} {self.value} {self.right})"

def create_rule(rule_string):
    # Tokenize the rule string into meaningful components (numbers, operators, and keywords)
    tokens = re.findall(r"\(|\)|\w+|>=|<=|>|<|==|!=|AND|OR|\'[\w\s]+\'", rule_string)
    return build_ast(tokens)

def build_ast(tokens):
    stack = []
    operators = []

    def apply_operator():
        right = stack.pop()
        left = stack.pop()
        operator = operators.pop()
        stack.append(Node(node_type="operator", value=operator, left=left, right=right))

    precedence = {"OR": 1, "AND": 2, ">": 3, "<": 3, ">=": 3, "<=": 3, "==": 3, "!=": 3}

    for token in tokens:
        if token.isnumeric() or re.match(r"'[\w\s]+'", token) or re.match(r"[a-zA-Z_]+", token):
            stack.append(Node(node_type="operand", value=token))
        elif token in precedence:
            while operators and operators[-1] != '(' and precedence[operators[-1]] >= precedence[token]:
                apply_operator()
            operators.append(token)
        elif token == '(':
            operators.append(token)
        elif token == ')':
            while operators[-1] != '(':
                apply_operator()
            operators.pop()  # pop '('

    while operators:
        apply_operator()

    return stack[0]

def combine_rules(rules):
    if len(rules) == 1:
        return create_rule(rules[0])

    combined_root = None
    for rule_string in rules:
        rule_ast = create_rule(rule_string)
        if combined_root is None:
            combined_root = rule_ast
        else:
            combined_root = Node(node_type="operator", value="AND", left=combined_root, right=rule_ast)

    return combined_root

def evaluate_rule(ast, data):
    if ast.node_type == "operand":
        # Split operand into key, operator, and value
        match = re.split(r"(<=|>=|!=|==|>|<)", ast.value)

        if len(match) < 3:
            # Handle the case where match doesn't have enough elements
            print(f"Error: Invalid operand format in AST: '{ast.value}'")
            return False

        key = match[0].strip()  # The attribute, e.g., 'age' or 'department'
        operator = match[1].strip()  # The operator, e.g., '>', '==', etc.
        value = match[2].strip()  # The value, e.g., '30' or 'Sales'

        # Handle numeric or string comparison
        if key in data:
            data_value = data[key]
            if value.startswith("'") and value.endswith("'"):
                # It's a string value, remove quotes
                value = value[1:-1]
            else:
                # Convert value to float for numeric comparison
                try:
                    value = float(value)
                except ValueError:
                    pass  # In case the value cannot be converted to float

            # Perform the comparison
            if operator == ">":
                return data_value > value
            elif operator == "<":
                return data_value < value
            elif operator == ">=":
                return data_value >= value
            elif operator == "<=":
                return data_value <= value
            elif operator == "==":
                return data_value == value
            elif operator == "!=":
                return data_value != value
        else:
            print(f"Warning: '{key}' not found in provided data")
            return False  # If the key isn't found in the data
    else:
        # Evaluate the left and right subtree recursively
        left_val = evaluate_rule(ast.left, data)
        right_val = evaluate_rule(ast.right, data)

        # Apply the operator (AND/OR)
        if ast.value == "AND":
            return left_val and right_val
        elif ast.value == "OR":
            return left_val or right_val
    return False

# Define some sample rules
rule1 = "((age > 30 AND department == 'Sales') OR (age < 25 AND department == 'Marketing')) AND (salary > 50000 OR experience > 5)"
rule2 = "((age > 30 AND department == 'Marketing')) AND (salary > 20000 OR experience > 5)"

# Create individual rules
ast_rule1 = create_rule(rule1)
ast_rule2 = create_rule(rule2)

# Print ASTs
print("AST for Rule 1:", ast_rule1)
print("AST for Rule 2:", ast_rule2)

# Combine the rules
combined_ast = combine_rules([rule1, rule2])
print("Combined AST:", combined_ast)

# Evaluate rules
data = {"age": 35, "department": "Sales", "salary": 60000, "experience": 3}
result = evaluate_rule(combined_ast, data)
print("Evaluation Result for Data 1:", result)

# Try another dataset
data2 = {"age": 40, "department": "Marketing", "salary": 15000, "experience": 10}
result2 = evaluate_rule(combined_ast, data2)
print("Evaluation Result for Data 2:", result2)


AST for Rule 1: age
AST for Rule 2: age
Combined AST: (age AND age)
Error: Invalid operand format in AST: 'age'
Error: Invalid operand format in AST: 'age'
Evaluation Result for Data 1: False
Error: Invalid operand format in AST: 'age'
Error: Invalid operand format in AST: 'age'
Evaluation Result for Data 2: False


In [ ]:
# AST Node class
class ASTNode:
    def __init__(self, node_type, left=None, right=None, value=None):
        self.node_type = node_type  # "operator" or "operand"
        self.left = left  # Left child node
        self.right = right  # Right child node
        self.value = value  # Operand value (if it's an operand)

    def __repr__(self):
        if self.node_type == "operator":
            return f"({self.left} {self.value} {self.right})"
        else:
            return str(self.value)


In [ ]:
import re

class ASTNode:
    def __init__(self, node_type, left=None, right=None, value=None):
        self.node_type = node_type  # "operator" or "operand"
        self.left = left  # Left child node
        self.right = right  # Right child node
        self.value = value  # Operand value (if it's an operand)

    def __repr__(self):
        if self.node_type == "operator":
            return f"({self.left} {self.value} {self.right})"
        else:
            return str(self.value)

def create_rule(rule_string):
    # Tokenize the rule string based on operators, parentheses, and operands
    tokens = re.split(r'(\s+AND\s+|\s+OR\s+|\(|\))', rule_string)
    tokens = [token.strip() for token in tokens if token.strip()]

    # Recursive function to parse the tokenized expression
    def parse_expression(tokens):
        stack = []
        operator_stack = []

        while tokens:
            token = tokens.pop(0)

            if token == '(':
                # Parse a subexpression recursively
                sub_expr = parse_expression(tokens)
                stack.append(sub_expr)
            elif token == ')':

                break
            elif token in ['AND', 'OR']:

                operator_stack.append(token)
            else:
                # Handle operands (conditions like "age > 30")
                operand = ASTNode(node_type="operand", value=token)
                stack.append(operand)
            if len(stack) >= 2 and operator_stack:
                right = stack.pop()
                left = stack.pop()
                operator = operator_stack.pop()
                node = ASTNode(node_type="operator", left=left, right=right, value=operator)
                stack.append(node)

        return stack[0] if stack else None

    return parse_expression(tokens)

# Example usage
rule1 = "((age > 30 AND department = 'Sales') OR (age < 25 AND department = 'Marketing')) AND (salary > 50000 OR experience > 5)"
ast_rule1 = create_rule(rule1)
print(f"AST for rule1: {ast_rule1}")


AST for rule1: (((age > 30 AND department = 'Sales') OR (age < 25 AND department = 'Marketing')) AND (salary > 50000 OR experience > 5))


In [ ]:
def combine_rules(rules, operator="AND"):
    if not rules:
        return None
    combined_ast = rules[0]
    for rule in rules[1:]:
        combined_ast = ASTNode(node_type="operator", left=combined_ast, right=rule, value=operator)
    return combined_ast


rule2 = "((age > 30 AND department = 'Marketing')) AND (salary > 20000 OR experience > 5)"
ast_rule2 = create_rule(rule2)
combined_ast = combine_rules([ast_rule1, ast_rule2], operator="AND")
print(f"Combined AST: {combined_ast}")


Combined AST: ((((age > 30 AND department = 'Sales') OR (age < 25 AND department = 'Marketing')) AND (salary > 50000 OR experience > 5)) AND ((age > 30 AND department = 'Marketing') AND (salary > 20000 OR experience > 5)))


In [ ]:
def evaluate_node(node, data):
    if node.node_type == "operand":
        # Parse the operand (e.g., "age > 30")
        if ">" in node.value:
            attr, val = node.value.split(">")
            return data[attr.strip()] > int(val.strip())
        elif "<" in node.value:
            attr, val = node.value.split("<")
            return data[attr.strip()] < int(val.strip())
        elif "=" in node.value:
            attr, val = node.value.split("=")
            return data[attr.strip()] == val.strip().strip("'")
    elif node.node_type == "operator":
        if node.value == "AND":
            return evaluate_node(node.left, data) and evaluate_node(node.right, data)
        elif node.value == "OR":
            return evaluate_node(node.left, data) or evaluate_node(node.right, data)
    return False

def evaluate_rule(ast, data):
    return evaluate_node(ast, data)

# Example usage
data = {"age": 35, "department": "Sales", "salary": 60000, "experience": 3}
result = evaluate_rule(combined_ast, data)
print(f"Evaluation result: {result}")


Evaluation result: False


In [ ]:
rule1_ast = create_rule("age > 30 AND department = 'Sales'")
data1 = {"age": 35, "department": "Sales", "salary": 60000, "experience": 3}
print(evaluate_rule(rule1_ast, data1))  # Should return True

combined_rule_ast = combine_rules([rule1_ast, create_rule("salary > 50000 OR experience > 5")], operator="AND")
print(evaluate_rule(combined_rule_ast, data1))  # Should return True


True
True
